## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [173]:
import os
from dotenv import load_dotenv
load_dotenv() ## a loading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")

In [174]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

In [175]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Pritesh and I am a Chief AI Engineer")])

AIMessage(content="Hello Pritesh,\n\nIt's a pleasure to meet you!  That's a fantastic title.  \n\nAs a Chief AI Engineer, I imagine you have a lot on your plate. What are some of the most exciting projects you're working on at the moment? \n\nI'm always eager to learn more about the cutting edge of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 23, 'total_tokens': 100, 'completion_time': 0.14, 'prompt_time': 0.000155189, 'queue_time': 0.014529451, 'total_time': 0.140155189}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bc539c26-4340-4f53-a041-18c9d598643c-0', usage_metadata={'input_tokens': 23, 'output_tokens': 77, 'total_tokens': 100})

In [176]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Pritesh and I am a Chief AI Engineer"),
        AIMessage(content="Hello Pritesh! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You told me your name is Pritesh and that you are a Chief AI Engineer!  \n\nIs there anything else you'd like to tell me about your work or yourself? I'm happy to chat. 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 99, 'total_tokens': 147, 'completion_time': 0.087272727, 'prompt_time': 0.003460177, 'queue_time': 0.011050221, 'total_time': 0.090732904}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fd1e6065-d785-418c-a270-83b0ecb94e20-0', usage_metadata={'input_tokens': 99, 'output_tokens': 48, 'total_tokens': 147})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [177]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [178]:
config={"configurable":{"session_id":"chat1"}}

In [179]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Pritesh and I am a Data Engineer")],
    config=config
)

In [180]:
response.content

"Hello Pritesh,\n\nIt's nice to meet you! As a Chief AI Engineer, I'm sure you're doing some fascinating work.\n\nIs there anything specific you'd like to talk about today? I'm happy to answer questions, provide information, or even brainstorm ideas with you.  \n\nFor example, are you:\n\n* **Working on a particular AI project?**\n* **Interested in learning more about a specific AI technology?**\n* **Looking for insights into the latest trends in AI?**\n\n\n\nLet me know how I can be helpful! \n\n"

In [181]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Pritesh.  I remember that from our introduction! 😊  \n\nHow can I help you further?  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 161, 'total_tokens': 189, 'completion_time': 0.050909091, 'prompt_time': 0.00510041, 'queue_time': 0.009501528, 'total_time': 0.056009501}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c7f98714-05d8-41ae-beec-9705a8ce508e-0', usage_metadata={'input_tokens': 161, 'output_tokens': 28, 'total_tokens': 189})

In [182]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [183]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John, it's nice to meet you! 👋 \n\nIs there anything I can help you with today?  \n"

In [184]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John. 😊  \n\nI remember!  \n\nHow can I help you further?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [195]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [196]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Pritesh")]})

AIMessage(content='Hello Pritesh! \n\nNice to meet you.  \n\nHow can I help you today? 😄 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 31, 'total_tokens': 56, 'completion_time': 0.045454545, 'prompt_time': 0.000326609, 'queue_time': 0.014318731, 'total_time': 0.045781154}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a28d5380-c0ae-4907-a7f5-cfa55ef6f8f2-0', usage_metadata={'input_tokens': 31, 'output_tokens': 25, 'total_tokens': 56})

In [198]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [199]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Pritesh")],
    config=config
)

response

AIMessage(content="Hi Pritesh, it's nice to meet you!  \n\nI see you've already introduced yourself.  Is there anything you'd like to talk about or ask me?  I'm here to help! 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 132, 'total_tokens': 183, 'completion_time': 0.092727273, 'prompt_time': 0.005083091, 'queue_time': 0.009445069, 'total_time': 0.097810364}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b2a621f1-76c1-4e5b-b861-cc754c76476c-0', usage_metadata={'input_tokens': 132, 'output_tokens': 51, 'total_tokens': 183})

In [200]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"Your name is Pritesh!  \n\nIt seems like you're having a little fun with the question. 😉  Is there anything else I can help you with?\n"

In [201]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [202]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Pritesh")],"language":"Hindi"})
response.content

'नमस्ते Pritesh! 👋  \n\nमुझे आपसे मिलकर ख़ुशी हो रही है। \n\nआप मुझसे क्या पूछना चाहते हैं? 😊 \n'

In [203]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [204]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Pritesh")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते Pritesh!  😊\n\nमुझे समझ आया, आप Pritesh हैं! 👋 \n\n\n\n'

In [205]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)
repsonse.content

'नमस्ते Pritesh!  😊\n\nमुझे समझ आया, आप Pritesh हैं! 👋 \n\n\n\n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [217]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=45,  
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)       

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [219]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter('messages')|trimmer)
    | prompt
    | model
)

res = chain.invoke({
    'messages': messages + [HumanMessage(content='what math problem did i aks?')],
    'language':'English'
})
res.content

'You asked: "whats 2 + 2" \n\n\nI\'m always here to help with more math problems if you have them! 😊  \n'

In [223]:
## lets wrap this in the message history 
config = {'configurable':{'session_id':'chat02'}}
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key='messages'
)

In [225]:
response =  with_message_history.invoke(
    {
        'messages':messages+[HumanMessage(content='what math problem do i ask?')],
        'language':'English'
    },
    config=config
)

res.content

'You asked: "whats 2 + 2" \n\n\nI\'m always here to help with more math problems if you have them! 😊  \n'

In [226]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Pritesh and I am a Chief AI Engineer', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Pritesh,\n\nIt's nice to meet you! As a Chief AI Engineer, I'm sure you're doing some fascinating work.\n\nIs there anything specific you'd like to talk about today? I'm happy to answer questions, provide information, or even brainstorm ideas with you.  \n\nFor example, are you:\n\n* **Working on a particular AI project?**\n* **Interested in learning more about a specific AI technology?**\n* **Looking for insights into the latest trends in AI?**\n\n\n\nLet me know how I can be helpful! \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 125, 'prompt_tokens': 23, 'total_tokens': 148, 'completion_time': 0.227272727, 'prompt_time': 0.00259811, 'queue_time': 0.02514765, 'total_time': 0.229870837}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', '